
#Importando as bibliotecas


In [ ]:
#1
import pandas as pd
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import numpy as np
import networkx as nx
!pip install plotly.express
from plotly import graph_objs as go

#2
!pip install fsspec
import urllib.parse
import pandas as pd
import requests
from io import StringIO

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


#Remoção de Stopwords




In [ ]:
# remoção de pontuacao e stopwords

def remove_stopwords(text,lang,domain_stopwords=[]):
  
  stop_words = nltk.corpus.stopwords.words(lang) # lang='portuguese' or lang='english'
  
  s = str(text).lower() # tudo para caixa baixa
  table = str.maketrans({key: None for key in string.punctuation})
  s = s.translate(table) # remove pontuacao
  tokens = word_tokenize(s) #obtem tokens
  v = [i for i in tokens if not i in stop_words and not i in domain_stopwords and not i.isdigit()] # remove stopwords
  s = ""
  for token in v:
    s += token+" "
  return s.strip()


# exemplos de uso
text = "Se você não sabe para onde ir, qualquer caminho serve."
text2 = remove_stopwords(text, 'portuguese')
print('Antes: '+text)
print('Depois: '+text2)

Antes: Se você não sabe para onde ir, qualquer caminho serve.
Depois: sabe onde ir qualquer caminho serve


#Radicalização


In [ ]:
def stemming(text,lang):
  
  stemmer = PorterStemmer() # stemming para ingles
  
  if lang=='portuguese':
    stemmer = nltk.stem.RSLPStemmer() # stemming para portuguese
    
  tokens = word_tokenize(text) #obtem tokens
  
  sentence_stem = ''
  doc_text_stems = [stemmer.stem(i) for i in tokens]
  for stem in doc_text_stems:
    sentence_stem += stem+" "
    
  return sentence_stem.strip()


# exemplos de uso
text = "Se você não sabe para onde ir, qualquer caminho serve."
text2 = remove_stopwords(text, 'portuguese')
text3 = stemming(text2, 'portuguese')
print('Antes: '+text)
print('Depois: '+text3)

Antes: Se você não sabe para onde ir, qualquer caminho serve.
Depois: sab ond ir qualqu caminh serv


#Analizando Dataset

In [ ]:
query = "qualquer coisa" # query para consultar na base de eventos (max. de 300 respostas)
query = urllib.parse.quote(query)

url = 'https://websensors.net.br/minicurso/2019/eventos-br-2017.php?q='+query
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
req = requests.get(url, headers=headers)

pd.set_option('display.max_colwidth', None)
dataset = pd.read_csv(StringIO(req.text),sep='\t', lineterminator='\n')

dataset[['event_id','date','title','local']]

,event_id,date,title,local
0,b1256cfabbd7e2046132a28662ef2990,2017-03-03 10:14:34,'Qualquer coisa é melhor que a Venezuela': os efeitos do deslocamento em massa em Boa Vista,Boa Vista (RR)
1,13a9834b6b98735cf1f934692dea67de,2017-03-03 10:54:00,'Qualquer coisa é melhor que na Venezuela': deslocamento em massa deixa Boa Vista em estado de emergência,Boa Vista (RR)
2,ff09e7cb07bc083525b3c228cb26dd59,2017-03-03 21:46:12,'Qualquer coisa é melhor que na Venezuela': deslocamento em massa deixa Boa Vista em estado de emergência,Boa Vista (RR)
3,36b0a55ff0573cde45e53fcafd178124,2017-04-26 13:44:35,"Espero qualquer coisa vindo do São José, diz vice do Novo Hamburgo sobre injúria racial",Novo Hamburgo (RS)
4,84ac002cf7c31667d602dcc4a280aa33,2017-06-13 12:39:31,Vereador denuncia uso de banheiro feminino da UFF por ‘homens que se intitulam qualquer coisa diversa’,Niteroi (RJ)
5,5e359ce53c85549a304e7317c2c81bf6,2017-06-29 19:00:02,"Depois que trabalhei em Brasília, tiro qualquer coisa de letra, diz Monica Iozzi",Brasilia (DF)
6,318d55bdecece0eee7a7453948056e18,2017-07-13 16:52:18,"Trump muda tom sobre Acordo de Paris após encontro com Macron: ""Qualquer coisa pode acontecer""",Franca (SP)
7,be70b3373cc9fffc8d6b660b6237895d,2017-07-13 17:09:21,"Trump muda tom sobre Acordo de Paris após encontro com Macron: ""Qualquer coisa pode acontecer"" - R7",Franca (SP)
8,68da6ffcdb2c09c73d148ee3e98100f2,2017-07-13 17:09:21,"Trump muda tom sobre Acordo de Paris após encontro com Macron: ""Qualquer coisa pode acontecer"" - R7",Franca (SP)
9,bded02f7b3782a2fd6cf72781d9d5060,2017-07-13 17:09:21,"Trump muda tom sobre Acordo de Paris após encontro com Macron: ""Qualquer coisa pode acontecer"" - R7",Franca (SP)


#Bag-of-Words

In [ ]:
def compute_bag_of_words(dataset,lang,domain_stopwords=[]):
  
  d = []
  for index,row in dataset.iterrows():
    text = row['title'] #texto do evento
    text2 = remove_stopwords(text, lang,domain_stopwords)
    text3 = stemming(text2, lang)
    d.append(text3)
  
  matrix = CountVectorizer(max_features=1000)
  X = matrix.fit_transform(d)
  
  count_vect_df = pd.DataFrame(X.todense(), columns=matrix.get_feature_names())

  return count_vect_df


bow = compute_bag_of_words(dataset,'portuguese')
bow

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,acontec,acord,aguent,alckmin,ant,após,banh,bi,boa,brasil,brasíl,brig,campeã,candidat,cas,cois,deix,denunc,desloc,divers,diz,dor,efeit,emerg,en,encontr,espaç,esper,est,far,feminin,globocom,hamburg,hom,injúr,intitul,iozz,josé,letr,macron,marid,mass,medin,melhor,mir,monic,mud,nov,paril,penich,pod,prov,qualqu,r7,rac,recif,ser,sobr,tir,tom,top,trabalh,trump,tud,uff,uso,venezuel,vere,vic,vind,vist
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
6,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
7,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
8,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
9,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0


#Ponderação de termos por TFIDF

In [ ]:
def compute_vsm_tfidf(dataset,lang,domain_stopwords=[]):
  
  d = []
  for index,row in dataset.iterrows():
    text = row['title'] #texto do evento
    text2 = remove_stopwords(text, lang,domain_stopwords)
    text3 = stemming(text2, lang)
    d.append(text3)
  
  matrix = TfidfVectorizer()
  X = matrix.fit_transform(d)
  
  tfidf_vect_df = pd.DataFrame(X.todense(), columns=matrix.get_feature_names())

  return tfidf_vect_df


vsm = compute_vsm_tfidf(dataset,'portuguese')
vsm

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,acontec,acord,aguent,alckmin,ant,após,banh,bi,boa,brasil,brasíl,brig,campeã,candidat,cas,cois,deix,denunc,desloc,divers,diz,dor,efeit,emerg,en,encontr,espaç,esper,est,far,feminin,globocom,hamburg,hom,injúr,intitul,iozz,josé,letr,macron,marid,mass,medin,melhor,mir,monic,mud,nov,paril,penich,pod,prov,qualqu,r7,rac,recif,ser,sobr,tir,tom,top,trabalh,trump,tud,uff,uso,venezuel,vere,vic,vind,vist
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.351274,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.155930,0.000000,0.000000,0.351274,0.000000,0.000000,0.000000,0.459357,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.351274,0.000000,0.351274,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.155930,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.351274,0.000000,0.000000,0.000000,0.351274
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.312971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.138927,0.352938,0.000000,0.312971,0.000000,0.000000,0.000000,0.000000,0.352938,0.000000,0.000000,0.000000,0.000000,0.352938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.312971,0.000000,0.312971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.138927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.312971,0.000000,0.000000,0.000000,0.312971
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.312971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.138927,0.352938,0.000000,0.312971,0.000000,0.000000,0.000000,0.000000,0.352938,0.000000,0.000000,0.000000,0.000000,0.352938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.312971,0.000000,0.312971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.138927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.312971,0.000000,0.000000,0.000000,0.312971
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.113453,0.000000,0.000000,0.000000,0.000000,0.255584,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.334223,0.000000,0.000000,0.000000,0.000000,0.334223,0.000000,0.334223,0.000000,0.000000,0.334223,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.334223,0.000000,0.000000,0.000000,0.000000,0.113453,0.000000,0.334223,0.000000,0.000000,0.209582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.288222,0.334223,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.329146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111730,0.000000,0.329146,0.000000,0.329146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.329146,0.000000,0.000000,0.329146,0.000000,0.329146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111730,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.329146,0.329146,0.000000,0.329146,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.383054,0.000000,0.000000,0.000000,0.000000,0.130029,0.000000,0.000000,0.000000,0.000000,0.292924,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.383054,0.000000,0.383054,0.000000,0.000000,0.000000,0.000000,0.000000,0

#Similaridade cosseno entre dois eventos na representação BoW

In [ ]:
def dis_cosine(matrix, e1, e2):
  dcos = cosine(matrix.iloc[e1,:], matrix.iloc[e2,:])
  return dcos


# exemplo: dissimilaride entre o primeiro (id=0) e o segundo evento (id=1) do vsm-tfidf
dis_cosine(vsm,0,1)

0.2970425331090243

#Gerando uma Rede de Documentos

In [ ]:
def compute_network(matrix,seed=0,min_dcos=0.7,max_neighbors=3,max_nodes=1000):
  
  G=nx.Graph()
  visited = []
  visited.append(seed)
  total = matrix.shape[0]
  temp = {}
  
  while(True):
  
    seed = visited.pop()
    temp[seed]=1
  
    neighbors = {}

    for i in range(0,total):
      if seed==i: continue
      if i in temp: continue
      dcos = dis_cosine(matrix,i,seed)
      if dcos <= min_dcos:
        neighbors[i] = dcos

    sorted_x = sorted(neighbors.items(), key=lambda kv: kv[1])

    counter=0
    for item in sorted_x:
      G.add_edge(seed, item[0], weight=(1-item[1]))
      if item[0] not in temp: visited.append(item[0])
      counter+=1
      if (counter >= max_neighbors): break     

  
    if(len(G) >= max_nodes): break
    if(len(visited)==0): break
    
    
    
  return G
    
G = compute_network(vsm,min_dcos=0.8,seed=1)
nx.info(G)

'Graph with 3 nodes and 3 edges'

#Visualizando

In [ ]:
def plot_event_network():
  
  # plotando rede
  pos = nx.drawing.layout.spring_layout(G)
  for item in pos:
    G.nodes[item]['pos']=[pos[item][0],pos[item][1]]
    
  edge_x = []
  edge_y = []
  for edge in G.edges():
      x0, y0 = G.nodes[edge[0]]['pos']
      x1, y1 = G.nodes[edge[1]]['pos']
      edge_x.append(x0)
      edge_x.append(x1)
      edge_x.append(None)
      edge_y.append(y0)
      edge_y.append(y1)
      edge_y.append(None)

  edge_trace = go.Scatter(
      x=edge_x, y=edge_y,
      line=dict(width=0.5, color='#888'),
      hoverinfo='none',
      mode='lines')

  node_x = []
  node_y = []
  for node in G.nodes():
      x, y = G.nodes[node]['pos']
      node_x.append(x)
      node_y.append(y)

  node_trace = go.Scatter(
      x=node_x, y=node_y,
      mode='markers',
      hoverinfo='text',
      marker=dict(
          showscale=True,
          # colorscale options
          #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
          #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
          #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
          colorscale='YlGnBu',
          reversescale=True,
          color=[],
          size=10,
          colorbar=dict(
              thickness=15,
              title='Node Connections',
              xanchor='left',
              titleside='right'
          )))
  
  node_adjacencies = []
  node_text = []
  for node, adjacencies in enumerate(G.adjacency()):
      node_adjacencies.append(len(adjacencies[1]))

  for node in G.nodes():
    node_text.append(str(dataset.iloc[node,:]['title']))
      
  node_trace.marker.color = node_adjacencies
  node_trace.text = node_text
  
  fig = go.Figure(data=[edge_trace, node_trace],
               layout=go.Layout(
                  title='Network',
                  showlegend=False,
                  hovermode='closest',
                  margin=dict(b=20,l=5,r=5,t=40),
                  annotations=[ dict(
                      text="",
                      showarrow=False,
                      xref="paper", yref="paper",
                      x=0.005, y=-0.002 ) ],
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  fig.show()

plot_event_network()